In [ ]:
import cv2

import cvzone

import math

from ultralytics import YOLO

import numpy as np

import tensorflow as tf

from time import sleep

import RPi.GPIO as GPIO



print('GPIO setting done')

GPIO.setmode(GPIO.BCM)

GPIO.setup(26, GPIO.OUT)





cap = cv2.VideoCapture(0) 

cap.set(3, 1280) 

cap.set(4, 720) 



model = YOLO('best (1).pt')

predictive_model_deficiency = tf.keras.models.load_model('Deficiency_Model_Final.h5')

predictive_model_disease = tf.keras.models.load_model('Disease_Model.h5')





classNames = ["person", "bicycle", "car", "motorbike", "aeroplane", "bus", "train", "truck", "boat", "traffic light", "fire hydrant", "stop sign", "parking meter", "bench", "bird", "cat", "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra", "giraffe", "backpack", "umbrella", "handbag", "tie", "suitcase", "frisbee", "skis", "snowboard", "sports ball", "kite", "baseball bat", "baseball glove", "skateboard", "surfboard", "tennis racket", "bottle", "wine glass", "cup", "fork", "knife", "spoon", "bowl", "banana", "apple", "sandwich", "orange", "broccoli", "carrot", "hot dog", "pizza", "donut", "cake", "chair", "sofa", "pottedplant", "bed", "diningtable", "toilet", "tvmonitor", "laptop", "mouse", "remote", "keyboard", "cell phone", "microwave", "oven", "toaster", "sink", "refrigerator", "book", "clock", "vase", "scissors", "teddy bear", "hair drier", "toothbrush"]

class_labels_deficiency = ['-K', '-N', '-P', 'FN']

class_labels_disease = ['Manioc_Bacteriose', 'Manioc_Healthy', 'Manioc_Mosaique', 'Manioc_Taches_Brunes']



alert_flag = False

GPIO.output(26, GPIO.HIGH)

while True:

    success, img = cap.read()

    

    if not success:

        print("Failed to read from camera")

        break  # Exit the loop if reading from the camera fails

    

    img = cv2.flip(img,1)

    results = model(img, stream=True)

    for r in results:

        boxes = r.boxes

        for box in boxes:

            x1, y1, x2, y2 = box.xyxy[0]

            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)

            w, h = x2 - x1, y2 - y1

            cvzone.cornerRect(img, (x1, y1, w, h))



            conf = math.ceil((box.conf[0] * 100)) / 100



            cls = box.cls[0]

            name = classNames[int(cls)]



            cvzone.putTextRect(img, f'{name} 'f'{conf}', (max(0, x1), max(35, y1)), scale=0.5)

            # cvzone.putTextRect(img, f'{name} 'f'{conf}', (max(0, x1), max(35, y1)), scale=0.5)

            

            # Calculate the center region based on frame size

            frame_height, frame_width, _ = img.shape

            center_x_min = int(frame_width * 0.4)

            center_x_max = int(frame_width * 0.6)

            center_y_min = int(frame_height * 0.4)

            center_y_max = int(frame_height * 0.6)



            # Check if the object is near the dynamically calculated center

            if center_x_min < (x1 + x2) / 2 < center_x_max and center_y_min < (y1 + y2) / 2 < center_y_max:

                alert_flag = True

            else:

                alert_flag = False



    if alert_flag:

        cv2.putText(img, "Object in Center!", (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)

        print("Object in Center!")

        

        '''Stop-delay'''

        GPIO.output(26, GPIO.LOW)

        cv2.imwrite("PlantImage.jpg", img)

        sleep(5)

        GPIO.output(26, GPIO.HIGH)



        PImage = cv2.imread("PlantImage.jpg")

        PImage = cv2.resize(PImage,(224, 224))

        PImage = np.reshape(PImage,[1,224,224,3])



        #For Deficiency Prediction

        prediction = predictive_model_deficiency.predict(PImage)



        if(class_labels_deficiency[prediction.argmax(axis=1)[0]] == '-K'):

            print("Plant is potassium deficit")



        elif(class_labels_deficiency[prediction.argmax(axis=1)[0]] == '-N'):

            print("Plant is nitrogen deficit")



        elif(class_labels_deficiency[prediction.argmax(axis=1)[0]] == '-P'):

            print("Plant is phosphorus deficit")



        else:

            print("Plant is healthy")



        #For Disease Prediction

        prediction = predictive_model_disease.predict(PImage)



        print(class_labels_disease[prediction.argmax(axis=1)[0]])

        #break

        #imshow opens only when object detected

        #cv2.imshow("Image", img)

        # Exit if the user presses 'q'

        #key = cv2.waitKey(1)

        #if key == ord('q'):

          # break

        ''' print('awake')'''

    #Window will break only by pressing q

    cv2.imshow("Image", img)



    # Exit if the user presses 'q'

    key = cv2.waitKey(1)

    if key == ord('q'):

       break



cap.release()

cv2.destroyAllWindows()

